## Combinação de variáveis para identificação de meios de transporte

A maior dificuldade dessa tarefa é diferenciar carros de ônibus e de táxis, tanto que em vários trabalhos essas categorias são unificadas em uma só. A partir da identificação de stay-points surgiu a ideia de relacionar as informações de quantidades de paradas e tempo de parada para facilitar nessa diferenciação, junto com as métricas de Fisher e de Jensen-Shannon.

Para a identificação dos pontos de parada será usada a biblioteca movingpandas. O tempo de cada parada pode ser obtido com a função get_stop_time_ranges() da classe TrajectoryStopDetector dessa biblioteca. Também pode ser possível incluir a variável tempo_entre_paradas.

In [2]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
import geopandas as gpd
from pyproj import CRS
from shapely.geometry import Point
from datetime import timedelta
import movingpandas as mpd
import matplotlib.pyplot as plt

In [ ]:
''' 
When reading all users, don't forget to check for the labels.txt file
'''
def read_one_user(n):
    user_path = '../Data/0'+str(n)+'/labels.txt'
    transport_df = pd.read_csv(user_path, sep="\t")
    
    directory = '../Data/0'+str(n)+'/Trajectory'
    
    gps_points_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'alt', 'identifier'])
    for filename in os.listdir(directory):
        dt_object = datetime.strptime(filename[:-4], '%Y%m%d%H%M%S')
        for index, row in transport_df.iterrows():
            start = datetime.strptime(row['Start Time'],'%Y/%m/%d %H:%M:%S')
            end = datetime.strptime(row['End Time'],'%Y/%m/%d %H:%M:%S')
            if dt_object >= start and dt_object <= end:
                file_df = pd.read_csv(directory+'/'+filename, skiprows=6, header=None,
                         parse_dates=[[5, 6]], infer_datetime_format=True)
                file_df.rename(inplace=True, columns={'5_6': 'time', 0: 'lat', 1: 'lon', 3: 'alt'})
                file_df.drop(inplace=True, columns=[2, 4])
                file_df['identifier'] = index
                file_df['geometry'] = [Point(lon, lat) for lon, lat in 
                             zip(file_df['lon'].to_list(), file_df['lat'].to_list())]
                gps_points_df = pd.concat([gps_points_df,file_df])
                
    return transport_df,gps_points_df

start = datetime.now()
transport_df,gps_points_df = read_one_user(10)

end = datetime.now()
print('Tempo gasto:',end-start)

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer w

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer w

In [ ]:
transport_df

In [ ]:
gps_points_df

In [ ]:
start = datetime.now()

gps_points_geometry_df = gps_points_df

# Creating a Geodataframe. Be aware it is CRS 4326 WGS84
# A GeoDataFrame object is a pandas.DataFrame that has a column with geometry
# It has one GeoSeries column that holds a special status. 
# This GeoSeries is referred to as the GeoDataFrame’s “geometry”. 
# A GeoSeries is essentially a vector where each entry in the vector is a set 
# of shapes corresponding to one observation.
geodata = gpd.GeoDataFrame(gps_points_geometry_df, crs = CRS.from_epsg('4326'))
geodata = geodata.set_index('time')

geo = datetime.now()
print('Tempo gasto criando geodata:',geo-start)

# Create a Trajectory Collection with Movingpandas
# data(list[Trajectory] or GeoDataFrame or DataFrame)–List of Trajectory 
#     objects or a GeoDataFrame with trajectory IDs, point geometry column 
#     and timestamp index
# traj_id_col(string)–Name of the GeoDataFrame column containing trajectory IDs
traj_collection = mpd.TrajectoryCollection(geodata, 'identifier')

traj = datetime.now()

print('Tempo gasto criando TrajectoryCollection:',traj-geo)

In [ ]:
start = datetime.now()

# Detects stops in a trajectory. A stop is detected if the movement stays 
# within an area of specified size for at least the specified duration.
# Define parameters in Hours and Search radius in meters
Hours = .01
SearchRadio = 500
stops = mpd.TrajectoryStopDetector(traj_collection).get_stop_segments(min_duration=timedelta(hours=Hours), max_diameter=SearchRadio)

end = datetime.now()
print('Tempo gasto detectando stop points:',end-start)

stops

In [ ]:
start = datetime.now()

Hours = .01
SearchRadio = 500
stops_duration = mpd.TrajectoryStopDetector(traj_collection).get_stop_time_ranges(min_duration=timedelta(hours=Hours), max_diameter=SearchRadio)

end = datetime.now()
print('Tempo gasto detectando stop point duration:',end-start)

stops_duration